In [1]:
import pandas as pd
import requests

from bs4 import BeautifulSoup

In [2]:
url = 'https://www.goldenglobes.com/winners-nominees/{year}?page={page}'

In [8]:
nominees = []

for i, year in enumerate(range(1944, 2021), start=1):
    page = 0
    req = requests.get(url.format(year=year, page=page))
    if req.status_code == 200:
        has_page = True
        
        soup = BeautifulSoup(req.content, 'html.parser')
        last_page = int(soup.find('li', class_='pager-current').text.split(' ')[-1]) - 1
            
        while has_page:
            award_groups = soup.find_all(class_='view-grouping')
            
            for award_group in award_groups:
                category = award_group.h2.text
                nominations = award_group.find_all('div', class_='views-row')
                
                for nominee in nominations:
                    if nominee.find(class_='views-field-field-nomination-is-winner').text.strip() == 'Winner':
                        win = True
                    else:
                        win = False
                        
                    nomination = nominee.find(class_='primary-nominee').text
                    film = nominee.find(class_='secondary-nominee')
                    
                    if film is not None:
                        film = film.text
                        
                    nominees.append([year - 1, year, i, category, nomination, film, win])
                    
            page += 1
            
            if last_page < page:
                has_page = False
                break
                
            print(year, page, last_page, has_page)
                
            req = requests.get(url.format(year=year, page=page))
            soup = BeautifulSoup(req.content, 'html.parser')
            
    else:
        print('Error: ', req.status_code)
        has_page = False
        break

1944 1 1 True
1944 2 1 True
1945 1 1 True
1945 2 1 True
1946 1 1 True
1946 2 1 True
1947 1 1 True
1947 2 1 True
1948 1 3 True
1948 2 3 True
1948 3 3 True
1948 4 3 True
1949 1 2 True
1949 2 2 True
1949 3 2 True
1950 1 3 True
1950 2 3 True
1950 3 3 True
1950 4 3 True
1951 1 3 True
1951 2 3 True
1951 3 3 True
1951 4 3 True
1952 1 4 True
1952 2 4 True
1952 3 4 True
1952 4 4 True
1952 5 4 True
1953 1 4 True
1953 2 4 True
1953 3 4 True
1953 4 4 True
1953 5 4 True
1954 1 3 True
1954 2 3 True
1954 3 3 True
1954 4 3 True
1955 1 4 True
1955 2 4 True
1955 3 4 True
1955 4 4 True
1955 5 4 True
1956 1 4 True
1956 2 4 True
1956 3 4 True
1956 4 4 True
1956 5 4 True
1957 1 5 True
1957 2 5 True
1957 3 5 True
1957 4 5 True
1957 5 5 True
1957 6 5 True
1958 1 4 True
1958 2 4 True
1958 3 4 True
1958 4 4 True
1958 5 4 True
1959 1 5 True
1959 2 5 True
1959 3 5 True
1959 4 5 True
1959 5 5 True
1959 6 5 True
1960 1 5 True
1960 2 5 True
1960 3 5 True
1960 4 5 True
1960 5 5 True
1960 6 5 True
1961 1 5 True
1961 2

In [9]:
df = pd.DataFrame(nominees, columns=['year_film', 'year_award', 'ceremony', 'category', 'nominee', 'film', 'win'])

In [22]:
df.loc[df.year_film == 2019]

,year_film,year_award,ceremony,category,nominee,film,win
7864,2019,2020,77,Best Motion Picture - Drama,1917,None,True
7865,2019,2020,77,Best Motion Picture - Drama,"Irishman, The",None,False
7866,2019,2020,77,Best Motion Picture - Drama,Joker,None,False
7867,2019,2020,77,Best Motion Picture - Drama,Marriage Story,None,False
7868,2019,2020,77,Best Motion Picture - Drama,"Two Popes, The",None,False
...,...,...,...,...,...,...,...
7986,2019,2020,77,Best Performance by an Actor in a Supporting R...,Kieran Culkin,Succession,False
7987,2019,2020,77,Best Performance by an Actor in a Supporting R...,Andrew Scott,Fleabag,False
7988,2019,2020,77,Best Performance by an Actor in a Supporting R...,Henry Winkler,Barry,False
7989,2019,2020,77,Cecil B. deMille Award,Tom Hanks,None,False


In [20]:
df['year_award'] = df.year_award.apply(lambda x: int(x) - 1)

In [23]:
df.to_csv('golden_globe_awards.csv', index=False, sep=',')